In [14]:
import pandas as pd
import pathlib
from pathlib import PurePath, Path
import numpy as np
import os
from tqdm import tqdm

In [15]:
def has_parent_X(path: Path, X: str) -> bool:
    for parent in path.parents:
        if parent.name == X:
            return True
    return False

In [29]:
# Define the base directory
data_dir = Path('training_datasets')

# Create a list of .fits files in the directory and its subdirectories
file_list = data_dir.glob("**/**/**/*.fits")

# Initialize the DataFrame with the desired columns
df = pd.DataFrame(columns=["file_name", "classes"])

# Function to check if a file has a specific parent directory
def has_parent_X(path, parent_dir):
    return parent_dir in path.parts

# Iterate through the list of files
for fl in tqdm(file_list):
    # Remove the 'training_datasets/' prefix from the path
    relative_path = fl.relative_to(data_dir).as_posix()

    # Determine the class based on the parent directory
    if has_parent_X(fl, "dual_AGN_datasets"):
        df.loc[len(df)] = {"file_name": relative_path, "classes": "dual_agn"}
    elif has_parent_X(fl, "single_AGN_datasets"):
        df.loc[len(df)] = {"file_name": relative_path, "classes": "single_agn"}
    elif has_parent_X(fl, "merger_datasets"):
        df.loc[len(df)] = {"file_name": relative_path, "classes": "merger"}
    elif has_parent_X(fl, "offset_AGN_datasets"):
        df.loc[len(df)] = {"file_name": relative_path, "classes": "offset_agn"}
    else:
        df.loc[len(df)] = {"file_name": relative_path, "classes": "empty_space"}

95731it [04:10, 381.44it/s]


In [30]:
df

,file_name,classes
0,dual_AGN_datasets/HSC_I/simulated_double_AGN_i...,dual_agn
1,dual_AGN_datasets/HSC_I/simulated_double_AGN_i...,dual_agn
2,dual_AGN_datasets/HSC_I/simulated_double_AGN_i...,dual_agn
3,dual_AGN_datasets/HSC_I/simulated_double_AGN_i...,dual_agn
4,dual_AGN_datasets/HSC_I/simulated_double_AGN_i...,dual_agn
...,...,...
95726,single_AGN_datasets/confirmed_single_AGN_sprin...,single_agn
95727,single_AGN_datasets/confirmed_single_AGN_sprin...,single_agn
95728,single_AGN_datasets/confirmed_single_AGN_sprin...,single_agn
95729,single_AGN_datasets/confirmed_single_AGN_sprin...,single_agn


In [31]:
import pandas as pd
# Count the occurrences of each class
class_counts = df['classes'].value_counts()

# Find the minimum count
min_count = class_counts.min()

# Sample min_count instances from each class
balanced_df = df.groupby('classes').apply(lambda x: x.sample(min_count)).reset_index(drop=True)

print(balanced_df)

                                               file_name     classes
0      dual_AGN_datasets/HSC_I/simulated_double_AGN_i...    dual_agn
1      dual_AGN_datasets/train_data/12_with_AGN_1179....    dual_agn
2      dual_AGN_datasets/train_data/14_with_AGN_655.fits    dual_agn
3       dual_AGN_datasets/train_data/8_with_AGN_902.fits    dual_agn
4      dual_AGN_datasets/HSC_I/simulated_double_AGN_i...    dual_agn
...                                                  ...         ...
19251  single_AGN_datasets/confirmed_single_AGN_fall/...  single_agn
19252  single_AGN_datasets/confirmed_single_AGN/QSO_1...  single_agn
19253  single_AGN_datasets/confirmed_single_AGN_sprin...  single_agn
19254  single_AGN_datasets/confirmed_single_AGN_fall/...  single_agn
19255  single_AGN_datasets/confirmed_single_AGN/UNK_0...  single_agn

[19256 rows x 2 columns]


In [32]:
balanced_df.to_csv("info.csv")